<a href="https://colab.research.google.com/github/Srinivaskolli45/-EVA8-Assignment/blob/main/S4/assignment4_step_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms    

  

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #input 1x28x28   ==> Output 10x26x26  | RF 3
        self.conv1 = nn.Sequential(nn.Conv2d(1, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      
        #input 10x26x26 ==> Output 10x24x24 | RF 5
        self.conv2 = nn.Sequential(nn.Conv2d(10, 10, 3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      
        #input 10x24x24 ==> Output 10x12x12 | RF  7
        self.transitionblk1 = nn.MaxPool2d(2,2)

        #input 10x12x12 ==> Output 32x10x10 | RF 10
        self.conv3 = nn.Sequential(nn.Conv2d(10,32,3),
                                      nn.BatchNorm2d(32),
                                      nn.ReLU())
                                     

        #input 32x10x10 ==> Output 10x8x8 | RF 14
        self.conv4 = nn.Sequential(nn.Conv2d(32,10,3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      
         
                                      
        #input 10x8x8 ==> Output 20x6x6 | RF 18
        self.conv5 = nn.Sequential(nn.Conv2d(10,20,3),
                                      nn.BatchNorm2d(20),
                                      nn.ReLU())
                                      
        #input 20x6x6 ==> OUtput 10x6x6 | RF 20
        self.conv6 = nn.Sequential(nn.Conv2d(20,10,1),
                                    nn.BatchNorm2d(10),
                                     nn.ReLU())
                                     
       
        
        self.avgPoolblk = nn.AvgPool2d(6,6)
    def forward(self, x):
        x = self.conv1(x)  #input 1x28x28   ==> Output 10x26x26  | RF 3
        x = self.conv2(x)  #input 10x26x26   ==> Output 10x24x24  | RF 5      
        x = self.transitionblk1(x) #input 10x24x24 ==> Output 10x12x12 | RF  6
        x = self.conv3(x)  #input 10x12x12 ==> Output 32x10x10 | RF 10
        x = self.conv4(x)   #input 32x10x10 ==> Output 10x8x8 | RF 14
        x = self.conv5(x)   #input 10x8x8 ==> Output 20x6x6 | RF 18
        x = self.conv6(x)        #input 20x6x6 ==> Output 10x6x6 | RF 18
        x = self.avgPoolblk(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)  


In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model,input_size=(1,28,28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
         MaxPool2d-7           [-1, 10, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           2,912
       BatchNorm2d-9           [-1, 32, 10, 10]              64
             ReLU-10           [-1, 32, 10, 10]               0
           Conv2d-11             [-1, 10, 8, 8]           2,890
      BatchNorm2d-12             [-1, 10, 8, 8]              20
    

<ipython-input-3-a70b35e0ffd3>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
import torchvision


torch.manual_seed(1)
batch_size = 128

#kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
kwargs = dict(num_workers= 4, pin_memory= True if use_cuda else False,shuffle=True,batch_size=batch_size)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomRotation(degrees=7)
                    ])),**kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #transforms.RandomRotation(degrees=7)
                    ])),**kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_loss = 0
    correct = 0
    processed=0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss +=loss
        pred = output.argmax(dim=1,keepdim=True)
        
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accurary={100*correct/processed:0.2f}')

    
   # print('\nTrain set: average loss: {:.4f}, accuracy: {}/{} ({:.0f}%)\n'.format(
   
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
schedular = StepLR(optimizer=optimizer,step_size=6,gamma=0.1)
for epoch in range(15):
    print(f'Epoch--> {epoch+1}')
    train(model, device, train_loader, optimizer, epoch)
    print("Current learning rate:", optimizer.param_groups[0]['lr'])
    schedular.step()
    test(model, device, test_loader)
     

Epoch--> 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-a70b35e0ffd3>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.060994941741228104 batch_id=468 Accurary=95.05: 100%|██████████| 469/469 [00:33<00:00, 14.06it/s]

Current learning rate: 0.2



Test set: Average loss: 0.0715, Accuracy: 9784/10000 (97.84%)

Epoch--> 2


loss=0.1192563995718956 batch_id=468 Accurary=98.30: 100%|██████████| 469/469 [00:33<00:00, 13.86it/s]

...

Current learning rate: 0.0020000000000000005



Test set: Average loss: 0.0183, Accuracy: 9948/10000 (99.48%)

Epoch--> 15


loss=0.007666804362088442 batch_id=468 Accurary=99.45: 100%|██████████| 469/469 [00:34<00:00, 13.43it/s]

Current learning rate: 0.0020000000000000005



Test set: Average loss: 0.0180, Accuracy: 9946/10000 (99.46%)

